# Exercícios em Python 6: Biblioteca padrão python, expressões regulares, csv, xml



## Preparando o ambiente

Execute o código a seguir.

In [1]:
!git clone https://gitlab.uspdigital.usp.br/thiago/curso_ai_python.git
import sys
sys.path.append('curso_ai_python')

Cloning into 'curso_ai_python'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 92 (delta 29), reused 0 (delta 0), pack-reused 33 (from 1)
Receiving objects: 100% (92/92), 28.45 KiB | 277.00 KiB/s, done.
Resolving deltas: 100% (45/45), done.


# Se o bloco acima foi executado corretamente, você deve ver o contéudo do diretório ```curso_ai_pytho`/dados```:

In [2]:
!ls -la curso_ai_python/dados/

total 32
drwxr-xr-x 2 root root  4096 May 19 13:35 .
drwxr-xr-x 5 root root  4096 May 19 13:35 ..
-rw-r--r-- 1 root root    20 May 19 13:35 dados01.txt
-rw-r--r-- 1 root root    50 May 19 13:35 dados02.txt
-rw-r--r-- 1 root root 12633 May 19 13:35 telefones_epusp.txt


## 6.1 Leitura de dados em CSV

O módulo ```csv``` em Python lê arquivos em formato CSV.

A função ```csv.DictReader(arquivo)``` retorna uma lista de dicionários, um para cada linha do arquivo.

Por exemplo, o código a seguir mostra o total de alunos estrangeiros no município de Araraquara.

In [3]:
import urllib.request
import csv
import io
import tempfile
import ssl

# Suprime verificação de SSL por problemas
#  com a ICP Brasil
ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')
total = 0
for entrada in leitorcsv:
  if entrada['MUN']=='ARARAQUARA':
    total += int(entrada[' Nº ALUNOS '])
print(total)

50


In [4]:
entrada

{'NOMEDEP': 'ESTADUAL - SE',
 'DE': 'VOTUPORANGA',
 'DISTR': 'VOTUPORANGA',
 'MUN': 'VOTUPORANGA',
 'TIPOESC': '8',
 'COD_ESC': '44167',
 'NOMESC': 'ENNY TEREZA LONGO FRACARO PROFA',
 'DS_PAIS': 'Venezuela',
 ' Nº ALUNOS ': ' 9 '}

### Exercício 6.1.1

Encontre o total de alunos provenientes do Uruguai matriculados em escolas de SP.
O campo que contém o país é o campo com nome ```DS_PAIS```.


In [5]:
import urllib.request
import csv
import io
import ssl

# Suprime verificação de SSL por problemas com a ICP Brasil
ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso, encoding='utf-8'), delimiter=';')

total = 0
for entrada in leitorcsv:
    if entrada['DS_PAIS'] == 'Uruguai':  # Ajuste para filtrar pelo país
        total += int(entrada[' Nº ALUNOS '])

print(total)

18


A resposta deve ser 18

### Exercício 6.1.2

Quantos *distritos* distintos de escolas existem nesta lista? O campo que contém o tipo é ```DISTR```.
Qual o tipo com a maior quantidade de alunos?

Sugestão: Crie um dicionário.


In [6]:

import pandas as pd


recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso, encoding='utf-8'), delimiter=';')

# Cria o DataFrame diretamente do DictReader
df = pd.DataFrame(leitorcsv)


# Exemplo: Calcular a quantidade de alunos por distrito
alunos_por_distrito = df.groupby('DISTR')[' Nº ALUNOS '].transform(lambda x: pd.to_numeric(x, errors='coerce')).groupby(df['DISTR']).sum()
print("\nQuantidade de alunos por distrito:")
print(alunos_por_distrito)

# Exemplo: Quantidade de distritos distintos
num_distritos_distintos = alunos_por_distrito.nunique()
print(f"\nQuantidade de distritos distintos: {num_distritos_distintos}")

# Exemplo: Distrito com a maior quantidade de alunos
distrito_mais_alunos = alunos_por_distrito.idxmax()
maior_quantidade = alunos_por_distrito.max()
print(f"\nDistrito com a maior quantidade de alunos: {distrito_mais_alunos} com {maior_quantidade} alunos")


Quantidade de alunos por distrito:
DISTR
ADAMANTINA                 11
AGUA RASA                 131
AGUAI                       1
AGUAS DE LINDOIA            1
AGUAS DE SANTA BARBARA      1
                         ... 
VILA SONIA                 18
VILA XAVIER                10
VINHEDO                    11
VOTORANTIM                 35
VOTUPORANGA                19
Name:  Nº ALUNOS , Length: 419, dtype: int64

Quantidade de distritos distintos: 115

Distrito com a maior quantidade de alunos: VILA MARIA com 642 alunos


A resposta deve ser 419 distritos distintos. O tipo com a maior quantidade é o ```VILA MARIA``` com 642 alunos.

## 6.2 Processamento de XML

A url ```https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='04-23-2020'&$top=100&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao``` recupera o relatório de cotação do dólar do Banco central em formato xml e grava-o no arquivo cotacao_dolar.xml.



In [7]:
req=urllib.request.urlopen("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2024'&@dataFinalCotacao='05-03-2024'&$top=200&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao")
ff = open("cotacao_dolar.xml", "wb")
ff.write(req.read())
ff.close()



O código abaixo usa xml para encontrar o valor da cotação no dia 17/03/2024

In [8]:
import xml.etree.ElementTree
raiz = xml.etree.ElementTree.parse("cotacao_dolar.xml").getroot()
for el in raiz.iter('{http://docs.oasis-open.org/odata/ns/metadata}properties'):
  hora = el.find('{http://docs.oasis-open.org/odata/ns/data}dataHoraCotacao')
  if hora.text.startswith('2024-05-02'):
    no_venda = el.find('{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda')
    valor = float(no_venda.text)
    print(valor)



5.1184


Exercício:

Encontre o *menor* valor de cotação do dólar para venda (campo com etiqueta ```{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda'```).

Você deve encontrar 4.8543

In [9]:
import urllib.request
import pandas as pd
import xml.etree.ElementTree as ET

# URL para obter a cotação do dólar no período especificado em XML
url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2024'&@dataFinalCotacao='05-03-2024'&$top=200&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"

# Faz a requisição e lê o conteúdo XML
req = urllib.request.urlopen(url)
xml_content = req.read()

# Salva o conteúdo XML em um arquivo (opcional, mas útil para inspeção)
with open("cotacao_dolar.xml", "wb") as f:
    f.write(xml_content)

# Analisa o XML usando ElementTree
root = ET.fromstring(xml_content)

# Define os namespaces para facilitar a busca
namespaces = {
    'md': 'http://docs.oasis-open.org/odata/ns/metadata',
    'd': 'http://docs.oasis-open.org/odata/ns/data'
}

# Extrai os dados relevantes para uma lista de dicionários
data = []
for entry in root.findall('.//md:properties', namespaces):
    cotacao_compra = entry.find('d:cotacaoCompra', namespaces).text
    cotacao_venda = entry.find('d:cotacaoVenda', namespaces).text
    data_hora_cotacao = entry.find('d:dataHoraCotacao', namespaces).text
    data.append({
        'cotacaoCompra': float(cotacao_compra),
        'cotacaoVenda': float(cotacao_venda),
        'dataHoraCotacao': data_hora_cotacao
    })

# Cria o DataFrame pandas
df_cotacao = pd.DataFrame(data)

# Encontra o menor valor da cotação de venda
menor_cotacao_venda = df_cotacao['cotacaoVenda'].min()

print("DataFrame com as cotações do dólar:")
print(df_cotacao)
print(f"\nO menor valor de cotação do dólar para venda é: {menor_cotacao_venda}")

DataFrame com as cotações do dólar:
    cotacaoCompra  cotacaoVenda          dataHoraCotacao
0          4.8910        4.8916  2024-01-02 13:05:50.319
1          4.9206        4.9212  2024-01-03 13:14:41.153
2          4.9182        4.9188  2024-01-04 13:05:35.048
3          4.8893        4.8899  2024-01-05 13:10:31.843
4          4.8844        4.8850   2024-01-08 13:11:30.76
..            ...           ...                      ...
80         5.1178        5.1184  2024-04-26 15:44:48.581
81         5.1149        5.1155  2024-04-29 13:08:20.023
82         5.1712        5.1718  2024-04-30 13:09:37.423
83         5.1178        5.1184  2024-05-02 13:07:09.245
84         5.0662        5.0668  2024-05-03 13:06:33.607

[85 rows x 3 columns]

O menor valor de cotação do dólar para venda é: 4.8543


A resposta deve ser 4.8543

## 5.3  Expressões Regulares

A linguagem Python tem uma biblioteca de expressões regulares.

A função ```re.search(exp, string)``` verifica se uma string é aderente a um determinado padrão.

### Exercícios:

O arquivo em ```/content/gdrive/My Drive/cursoai_python_aula_03/dados/telefones_epusp.txt``` contém uma lista telefônica em um texto não-estruturado com telefones úteis da escola politécnica.

Eis o conteúdo de suas 10 primeiras linhas:

In [10]:
arquivo = open("/content/curso_ai_python/dados/telefones_epusp.txt", "rt")
for i in range(10):
  print(arquivo.readline())
arquivo.close()

Diretoria

Profa. Dra. Liedi Legi Bariani Bernucci

Diretora 	3091 5221 / 5223 / 5550

Prof. Reinaldo Giudici

Vice-Diretor

Secretaria: Sueli Rossetti / Ângela de Jesus

Serviço Técnico de Planejamento e Gestão

Enaége Dalan Sant´Ana 	3091 5534

Seção Técnica de Qualidade e Processos

Patricia Sandes 	3091 5577



### 5.3.1 Conte o número de linhas com telefones

A expressão regular ```([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$``` casa com um ou mais números de telefone *ao final de uma linha*.

Verifique usando o módulo re quantas linhas atendem este padrão.
Você deve encontrar 255 números.

In [12]:
import re

# Abre o arquivo
arquivo = open("/content/curso_ai_python/dados/telefones_epusp.txt", "rt")

# Expressão regular
padrao = r"([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$"

# Contador
contador = 0

# Percorre cada linha e verifica se ela casa com o padrão
for linha in arquivo:
    if re.search(padrao, linha):
        contador += 1

arquivo.close()

# Exibe o resultado
print(f"Número de linhas com telefones: {contador}")


Número de linhas com telefones: 255
